<a href="https://colab.research.google.com/github/linyuehzzz/privacy_ga/blob/main/set_coverage_ga.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Model to reassign all true uniques: set coverage (genetic algorithm)**

In [126]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### **Reading individual data**

- Geographic level: tract
- Attributes: VOTINGAGE (2) $*$ HISPANIC (2) $*$ CENRACE (63)

In [127]:
%cd "/content/gdrive/My Drive/Colab Notebooks/privacy_ga"
import pandas as pd
import numpy as np

filename_hist = 'data/franklin_hist.csv'
hist = pd.read_csv(filename_hist)

# block to tract
hist['TRACT'] = hist['GEOID10'].astype(str).str[:11]
col_names = hist.columns.to_numpy()
col_names = np.delete(col_names, [0, -1])
hist = hist.groupby('TRACT').sum()[col_names]
hist = hist.reset_index()
hist

/content/gdrive/My Drive/Colab Notebooks/privacy_ga


,TRACT,00000000,00000001,00000002,00000003,00000004,00000005,00000006,00000007,00000008,00000009,00000010,00000011,00000012,00000013,00000014,00000015,00000016,00000017,00000018,00000019,00000020,00000021,00000022,00000023,00000024,00000025,00000026,00000027,00000028,00000029,00000030,00000031,00000032,00000033,00000034,00000035,00000036,00000037,00000038,...,07010123,07010124,07010125,07010126,07010127,07010128,07010129,07010130,07010131,07010132,07010133,07010134,07010135,07010136,07010137,07010138,07010139,07010140,07010141,07010142,07010143,07010144,07010145,07010146,07010147,07010148,07010149,07010150,07010151,07010152,07010153,07010154,07010155,07010156,07010157,07010158,07010159,07010160,07010161,07010162
0,39049000110,438,6,0,7,0,4,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,39049000120,467,4,0,11,0,2,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,39049000210,430,11,0,6,0,4,4,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,39049000220,653,3,1,12,0,6,7,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,39049000310,313,205,0,17,0,2,19,0,4,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,39049010500,1821,35,1,409,0,2,5,2,22,0,1,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
280,39049010601,1408,36,0,146,0,3,12,0,15,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
281,39049010602,1585,45,3,139,0,4,7,2,9,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
282,39049010700,83,11,0,4,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Converting individual data

In [128]:
# HHGQ (8) $*$ VOTINGAGE (2) $*$ HISPANIC (2) $*$ CENRACE (63) to VOTINGAGE (2) $*$ HISPANIC (2) $*$ RACE (7)
n2, n3, n4 = 2, 2, 63

for y in range(n2):  # voting age
    y = '{number:0{width}d}'.format(width=2, number=y)
    col_names = [col for col in hist.columns if y in col[2:4] and len(col)==8]

    for z in range(n3):  # ethnicity
        z = '{number:0{width}d}'.format(width=2, number=z)
        col_names2 = [col for col in col_names if z in col[4:6]]

        col_two_or_more_races = []
        for x in range(n4):  # race
            if x >= 0 and x <= 5:
                x = '{number:0{width}d}'.format(width=2, number=x)
                col_names3 = [col for col in col_names2 if x in col[6:8]]
                hist[x + y + z] = hist[col_names3].sum(axis=1)
            else:
                x = '{number:0{width}d}'.format(width=2, number=x)
                col_names3 = [col for col in col_names2 if x in col[6:8]]
                col_two_or_more_races.extend(col_names3)
        hist[x + y + '06'] = hist[col_two_or_more_races].sum(axis=1)

hist.drop([col for col in hist.columns if len(col)==8], axis=1, inplace=True)
hist            

,TRACT,000000,010000,020000,030000,040000,050000,620006,000001,010001,020001,030001,040001,050001,000100,010100,020100,030100,040100,050100,620106,000101,010101,020101,030101,040101,050101
0,39049000110,438,6,0,7,0,4,1,29,0,1,0,0,1,2733,31,1,29,0,2,0,17,3,0,0,0,6
1,39049000120,467,4,0,11,0,2,0,13,0,2,0,0,1,2518,40,1,31,1,0,0,18,0,0,0,0,4
2,39049000210,430,11,0,6,0,4,0,10,0,0,0,0,0,2299,41,5,26,0,2,0,34,0,0,0,0,4
3,39049000220,653,3,1,12,0,6,5,20,0,0,0,0,4,2847,29,1,43,0,6,1,16,0,2,0,0,8
4,39049000310,313,205,0,17,0,2,3,15,1,0,0,0,18,1767,532,2,53,2,5,1,40,1,1,0,0,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,39049010500,1821,35,1,409,0,2,4,27,0,0,2,0,14,4581,73,2,768,1,8,0,70,0,1,0,0,19
280,39049010601,1408,36,0,146,0,3,2,16,3,0,0,0,14,2423,89,3,307,0,2,1,86,0,0,0,0,13
281,39049010602,1585,45,3,139,0,4,1,13,1,1,0,0,17,3817,109,3,281,0,8,0,71,2,2,0,0,25
282,39049010700,83,11,0,4,0,0,0,6,1,0,0,0,3,963,107,4,57,1,2,0,10,0,0,0,0,14


### **Defining parameters**

Defining global parameters

In [129]:
HIGH = 2   # binary variables
POOL_SIZE = 100
NGEN = 100
MUT_PROB = 0.9
TOURN_SIZE = 5
N, K = hist.shape[0], hist.shape[1] - 1

Defining $V_k$ and $w_{jk}$



In [130]:
import numpy as np

# define all the input data for the model   
V = {}
for k in range(K):
    V[k] = hist.index[hist.iloc[:, k+1] == 1]

count = 0
for key in V:
    count += len(V[key])
print(count)

W = hist.iloc[:, 1:].apply(lambda x: 1 + 1 / x).to_numpy()
W = np.nan_to_num(W, posinf=100) 
print(W.shape, W[0])

668
(284, 26) [  1.00228311   1.16666667 100.           1.14285714 100.
   1.25         2.           1.03448276 100.           2.
 100.         100.           2.           1.0003659    1.03225806
   2.           1.03448276 100.           1.5        100.
   1.05882353   1.33333333 100.         100.         100.
   1.16666667]


### **Coverage modeling**

Coverage I: all except the origin and other uniques

In [131]:
from collections import defaultdict

def cvgeAll(V, K, N):
    A = defaultdict(list)
    for k in range(K):
        for i in range(N):
            for j in range(N):
                if i != j and j not in V[k]:
                    A[(k, i)].append(j)
    return A

A = cvgeAll(V, K, N)
len(A)

7384

### **Initialization**

In [132]:
from collections import defaultdict
import numpy as np
from random import sample

def init(high, V, A):
    individual = defaultdict(list)
    for k in V:
        for i in V[k]:
            individual[k].append(sample(A[(k, i)], 1)[0])
    return individual

pop = np.array([init(HIGH, V, A) for i in range(POOL_SIZE)])
len(pop), pop[1][11], pop[2][11]

(100, [113, 63, 123, 216], [128, 130, 56, 69])

###**Evaluation**

In [133]:
def evaluate(individual, W):
    obj = 0 
    for k in individual:
        for j in individual[k]:
            obj += W[j, k]
    return obj

# obj = evaluate(pop[1], W)
# obj

###**Crossover**

One-point crossover

In [134]:
from random import randrange
import copy

def cxOnePoint(child1, child2):
    for k in child1:
        cxPoint = randrange(len(child1[k]) + 1)
        child1[k][:cxPoint], child2[k][:cxPoint] = child2[k][:cxPoint], child1[k][:cxPoint]
    return child1, child2

# child1, child2 = copy.deepcopy(pop[1]), copy.deepcopy(pop[2])
# print(child1[11], child2[11])
# child1, child2 = cxOnePoint(child1, child2)
# print(child1[11], child2[11])

Two-point crossover

In [135]:
from random import sample
import copy

def cxTwoPoint(child1, child2):
    for k in child1:
        cxPoints = sample(range(len(child1[k]) + 1), 2)
        child1[k][min(cxPoints):max(cxPoints)], child2[k][min(cxPoints):max(cxPoints)] = child2[k][min(cxPoints):max(cxPoints)], child1[k][min(cxPoints):max(cxPoints)]
    return child1, child2

# child1, child2 = copy.deepcopy(pop[1]), copy.deepcopy(pop[2])
# print(child1[11], child2[11])
# child1, child2 = cxTwoPoint(child1, child2)
# print(child1[11], child2[11])

Uniform crossover

In [136]:
from random import sample
from collections import defaultdict
import copy

def cxUniform(child1, child2):
    child1_new, child2_new = defaultdict(list), defaultdict(list)
    for k in child1:
        for idx, val in enumerate(child1[k]):
            child1_new[k].append(sample([val, child2[k][idx]], 1)[0])
            child2_new[k].append(sample([val, child2[k][idx]], 1)[0])
    return child1_new, child2_new

# child1, child2 = copy.deepcopy(pop[1]), copy.deepcopy(pop[2])
# print(child1[11], child2[11])
# child1, child2 = cxUniform(child1, child2)
# print(child1[11], child2[11])

###**Mutation**

Random resetting mutation

In [137]:
from random import random, sample

def mutRandomReset(mutant, prob, V, A):
    for k in mutant:
        for idx, val in enumerate(mutant[k]):
            if random() < prob:
                i = V[k][idx]
                mutant[k][idx] = sample(A[(k, i)], 1)[0]
    return mutant

# child = copy.deepcopy(pop[1])
# print(child[11])
# child = mutRandomReset(child, 0.6, V, A)
# print(child[11])

###**Selction**

Tournament selection

In [138]:
import math

def selTournament(population, size, t, W):
    population_new, obj_list = [], []

    for i in range(size):
        random_indices = np.random.choice(len(population), size=t, replace=False)
        group = population[random_indices,]

        obj, winner = math.inf, []
        for ind in group:
            curr_obj = evaluate(ind, W)
            if curr_obj < obj:
                winner = ind
                obj = curr_obj
        population_new.append(winner)
        obj_list.append(obj)

    return np.array(population_new), obj_list

t = 5
pop_new, obj_list = selTournament(pop, POOL_SIZE, t, W)
obj_list

[36779.180787323676,
 35200.67494475327,
 35597.74468348742,
 34905.68950692185,
 35200.67494475327,
 35200.67494475327,
 35597.74468348742,
 34905.68950692185,
 35402.96902532335,
 34909.28657243092,
 35500.743278909365,
 35402.87669927153,
 34610.99167162664,
 35995.7757703406,
 36289.81792569617,
 35798.366467494285,
 35108.2675046556,
 35597.675254702,
 36682.905380415104,
 35895.65096451075,
 35402.87669927153,
 36485.10553573365,
 35103.30196753793,
 35202.22997898533,
 36390.09291030907,
 34416.5048998257,
 35500.56011332015,
 35500.56011332015,
 36388.875196179404,
 35700.13250856528,
 35200.67494475327,
 35202.22997898533,
 35597.74468348742,
 35992.83806742973,
 35302.39301810916,
 35699.21872584647,
 34416.5048998257,
 35307.15350672303,
 35402.87669927153,
 35200.67494475327,
 35404.790538040324,
 35402.96902532335,
 34416.5048998257,
 35700.13250856528,
 35302.39301810916,
 35992.83806742973,
 34909.28657243092,
 35202.22997898533,
 35597.74468348742,
 35402.96902532335,
 

Selection the best t

In [139]:
import math
from collections import defaultdict

def selBest(population, size, t, W):
    population_new_dict = defaultdict(list)

    for idx, val in enumerate(population):
        obj = evaluate(val, W)
        population_new_dict[idx] = obj
    population_sorted = sorted(population_new_dict.items(), key=lambda x: x[1], reverse=False)

    population_new, obj_list = [], []
    for i in range(size):
        idx = population_sorted[i][0]
        population_new.append(population[idx])
        obj_list.append(population_sorted[i][1])

    return np.array(population_new), obj_list

t = 5
pop_new, obj_list = selBest(pop, POOL_SIZE, t, W)
obj_list

[34416.5048998257,
 34610.99167162664,
 34905.68950692185,
 34909.28657243092,
 35103.30196753793,
 35108.2675046556,
 35200.67494475327,
 35202.22997898533,
 35205.54038687517,
 35302.39301810916,
 35307.15350672303,
 35402.87669927153,
 35402.96902532335,
 35404.790538040324,
 35500.56011332015,
 35500.743278909365,
 35506.66767983239,
 35597.675254702,
 35597.74468348742,
 35697.61923752982,
 35698.52196170243,
 35699.21872584647,
 35700.13250856528,
 35796.23434956342,
 35798.366467494285,
 35800.0186706888,
 35895.65096451075,
 35992.83806742973,
 35995.624027248385,
 35995.7757703406,
 35997.617988207116,
 36092.48971370512,
 36289.81792569617,
 36291.42912221091,
 36388.875196179404,
 36389.13645082931,
 36389.70779873929,
 36389.711794087656,
 36390.09291030907,
 36485.10553573365,
 36485.95933103498,
 36489.08778119337,
 36583.33545302972,
 36583.96788563747,
 36584.372548392515,
 36587.58362739483,
 36589.31192789813,
 36682.905380415104,
 36684.07115202726,
 36684.1636243844

### **Main algorithm**

In [140]:
def main():
    A = cvgeAll(V, K, N)

    pop = np.array([init(HIGH, V, A) for i in range(POOL_SIZE)])

    for g in range(NGEN):
        for ind1, ind2 in zip(pop[::2], pop[1::2]):
            parent1, parent2 = copy.deepcopy(ind1), copy.deepcopy(ind2)
            child1_new, child2_new = cxUniform(parent1, parent2)
            pop = np.append(pop, child1_new)
            pop = np.append(pop, child2_new)
        
        for ind in pop:
            mutant = copy.deepcopy(ind)
            child_new = mutRandomReset(mutant, MUT_PROB, V, A)
            pop = np.append(pop, child_new)
        
        pop, objs = selBest(pop, POOL_SIZE, TOURN_SIZE, W)
    
        print(max(objs), min(objs), sum(objs)/len(objs))

    return 

pop = main()

35696.122350972626 33233.12207382526 35022.151782452056
34908.87568498774 33230.35927096955 34355.10277224579
34316.4864950062 31557.653067974297 33761.79440913417
33920.032504513576 31454.513907814275 33291.8974737847
33425.424656460404 30370.37903485327 32708.45343799999
32836.61969890036 30370.37903485327 32180.64113108049
32342.56961208759 29680.148907630413 31631.880445693136
31749.55465704367 29283.334879477083 30995.674771913145
31059.06811788591 29184.446130493827 30392.635761862624
30465.013049616413 27703.489006182925 29866.9603847692
29976.859034901252 27701.23216244559 29341.949056282312
29384.764109448166 26721.128955404594 28754.761453124418
28985.100878856472 25829.465052573236 28169.642955765314
28103.379345125755 25733.66058975656 27557.791884061004
27704.75573273957 25733.66058975656 27089.85910093355
27212.05148547121 25042.920230523247 26672.608995372037
26713.529958373452 25038.750794056872 26169.777140550235
26225.85854113046 24249.090658127676 25751.379303163365
